<a href="https://colab.research.google.com/github/valebl/SSZ_Clustering/blob/main/Catalogue_declustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CATALOGUE DECLUSTERING

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'
from copy import deepcopy
import seaborn as sns

from utils import haversine_np, is_leap_year, date_number

In [ ]:
# PATHS -> change if you need

input_catalogue_path = "/content/drive/MyDrive/SeismicSources/CATALOGUE.xlsx"
output_catalogue_path = "/content/drive/MyDrive/SeismicSources/CPTI15_v3.0_declustered.xlsx"

# FLAGS

write_csv = False

In [2]:
df = pd.read_excel(input_catalogue_path, usecols="A:M", engine="openpyxl")
df = df[['Year','Mo','Da','Ho','Mi','Se','LatDef','LonDef','DepDef','MwDef','X','Y']]

print(f"The database contains {len(df)} events\n")

df = df[df['LatDef'].notna()]
print(f"After removing LatDef=NaN: {len(df)} events")

df = df[df['LonDef'].notna()]
print(f"After removing also LonDef=NaN: {len(df)} events")

df = df[df['MwDef'].notna()]
print(f"After removing also MwDef=NaN: {len(df)} events")

df = df[df['Year'].notna()]
print(f"After removing also Year=NaN: {len(df)} events")

df = df[df['Mo'].notna()]
print(f"After removing also Mo=NaN: {len(df)} events")

df = df[df['Da'].notna()]
print(f"After removing also Da=NaN: {len(df)} events\n")

print(df.head(10), '\n')

print(f"Ho=Nan, Mi=NaN and Se=NaN are substituted with 0\n")
df[['Ho','Mi','Se']] = df[['Ho','Mi','Se']].replace([np.nan, "NaN"], 0)

print(df.head(10), '\n')


# Write the dataframe object into csv file
if write_csv:
    df.to_csv ("CPTI15_v3.0.csv", header=True)


The database contains 4860 events

After removing LatDef=NaN: 4748 events
After removing also LonDef=NaN: 4748 events
After removing also MwDef=NaN: 4703 events
After removing also Year=NaN: 4703 events
After removing also Mo=NaN: 4651 events
After removing also Da=NaN: 4597 events

    Year    Mo    Da    Ho  ...  DepDef  MwDef             X             Y
2   1019   4.0   1.0   NaN  ...     NaN   4.63  9.850866e+05  4.569418e+06
3   1044   4.0  19.0   9.0  ...     NaN   4.63  9.850866e+05  4.569418e+06
5   1065   3.0  27.0   NaN  ...     NaN   5.10  5.952487e+05  5.043553e+06
7   1087   9.0  10.0   NaN  ...     NaN   4.86  1.160782e+06  4.583025e+06
8   1091   1.0  27.0   NaN  ...     NaN   5.10  7.884312e+05  4.644411e+06
9   1094   1.0  14.0   NaN  ...     NaN   4.63  9.850866e+05  4.569418e+06
10  1117   1.0   3.0  15.0  ...     NaN   6.52  6.580732e+05  5.014586e+06
12  1120   3.0  25.0   NaN  ...     NaN   5.80  9.112262e+05  4.592288e+06
13  1125   6.0   7.0  11.0  ...     NaN  

## Declustering

The script works on a dataframe structured as follows:

Year  Month  Day  Hour  Minute  Second  Latitude  Longitude  Depth  Mw  X  Y

In [5]:
## Calculate the distance matrix
LONG_ALL = df['LonDef'].values
LAT_ALL = df['LatDef'].values

DISTANCES = [haversine_np(lon, lat, LONG_ALL, LAT_ALL) for lon, lat in zip(LONG_ALL,LAT_ALL)]

In [6]:
print(len(DISTANCES[0]))

4597


In [7]:
print(DISTANCES[0])

[  0.           0.         612.69046636 ... 426.61215296   6.57567579
 365.54154433]


In [8]:
## First screening of the catalog for the removal of the AFTERSHOCK
# Forward screening

MAGNITUDE_ALL = df['MwDef'].values
LONG_ALL = df['LonDef'].values
LAT_ALL = df['LatDef'].values

INDEX_DF = df.index.values

Year_all   = df['Year'].values
Month_all  = df['Mo'].values
Day_all    = df['Da'].values
Hour_all   = df['Ho'].values
Minute_all = df['Mi'].values
Second_all = df['Se'].values

Year_all = Year_all.astype('int')
Month_all = Month_all.astype('int')
Day_all = Day_all.astype('int')
Hour_all = Hour_all.astype('int')
Minute_all = Minute_all.astype('int')
Second_all = Second_all.astype('int')

ind_catalogue_aftershock = []

# CONVERSION OF ALL DATES TO NUMBERS
DateNumber_all = [date_number(Year_all[j],Month_all[j],Day_all[j],Hour_all[j],Minute_all[j],Second_all[j]) for j in range(len(df))]

for i in range(len(df)):
    # Extraction of the magnitude from the catalog

    Mw = MAGNITUDE_ALL[i]

    # Extraction of epicenter location from the catalog
    LONG = LONG_ALL[i]
    LAT  = LAT_ALL[i]
    
    # DEFINITION OF THE TIME WINDOW ACCORDING TO 3 DIFFERENT FORMULATIONS
    # The final time windows, in days, will be average of the three.
    if Mw >= 6.5:
        # Formulation according to (Gardner and Knopoff 1974)
        DT1 = 10**(0.032*Mw + 2.7389) # DAYS
        # Formulation according to (Gruenthal)
        DT2=abs(np.exp(-3.95+(0.62+17.32*Mw)**2)) # DAYS
    else:
        # Formulation according to (Gardner and Knopoff 1974)
        DT1 = 10**(0.5409*Mw - 0.547) # DAYS
        # Formulation according to (Gruenthal)
        DT2=10**(2.8+0.024*Mw) # DAYS

    # Formulation according to (Uhrhammer 1986)
    DT3 = np.exp(-2.87+1.235*Mw)
    
    # Average of the three different formluations
    DT = [DT1, DT2, DT3]
    DT = [0 if dt == np.inf else dt for dt in DT]
    DT = max(DT) # DAYS
    
    # DEFINITION OF THE SPATIAL WINDOW ACCORDING TO 3 DIFFERENT FORMULATIONS
    # The final time windows, in days, will be average of the three.
    # Formulation according to (Gardner and Knopoff 1974)
    R1 = 10**(0.1238*Mw+0.983) # km
    # Formulation according to (Gruenthal)
    R2 = np.exp(1.77+(0.037+1.02*Mw)**2) # km
    # Formulation according to (Uhrhammer 1986)
    R3 = np.exp(-1.024+0.804*Mw) # km
    
    # Average of the three different formluations
    R = [R1, R2, R3]
    R = [0 if r > 1000 else r for r in R]
    R = max(R) # km
    
    # CALCULATION OF THE DISTANCE BETWEEN THE CURRENT EPICENTER AND ALL THE
    # OTHER EPICENTERS OF THE OTHER EVENTS
    DISTANCES_i = DISTANCES[i]

    Year   = Year_all[i]
    Month  = Month_all[i]
    Day    = Day_all[i]
    Hour   = Hour_all[i]
    Minute = Minute_all[i]
    Second = Second_all[i]
    
    # CALCULATION OF THE CURRENT DAY + TIME WINDWOW
    y=np.floor(DT/365)
    m=np.floor(((DT/365-y)*365)/30)
    d=(((DT/365-y)*365)/30-m)*30

    months_days = [31 if i not in [3, 5, 8, 10] else 30 for i in range(12)]
    months_days[1] = 29 if is_leap_year(int(Year+y)) else 28

    while int(Month+m) > 12 or int(Day+d) > months_days[int(Month+m)-1]:
        
        if int(Month+m) > 12:
            y += 1
            m -= 12
            months_days[1] = 29 if is_leap_year(int(Year+y)) else 28

        if int(Day+d) > months_days[int(Month+m)-1]:
            d -=  months_days[int(Month+m)-1]
            m += 1

    UpperBoundTime = date_number(int(Year+y),int(Month+m),int(Day+d),Hour,Minute,Second)
    
    # IDENTIFICATION OF THE EVENTS FALLING IN THE TIME-SPACE WINDOW HAVING
    # A MAGNITUDE LOWER OR EQUAL THAN THE CURRENT VALUE Mw
    index = []
    [index.append(idx) for idx in range(i+1,len(df)) if DateNumber_all[idx]>DateNumber_all[i] and DateNumber_all[idx]<=UpperBoundTime and MAGNITUDE_ALL[idx]<Mw and DISTANCES_i[idx]<=R]
    
    # ASSEMBLING THE INDEXES IN A SINGLE MATRIX
    [ind_catalogue_aftershock.append(idx) for idx in index if index != []]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: overflow encountered in exp


In [9]:
# REMOVAL OF REPETING INDEXES
ind_catalogue_aftershock_unique = np.unique(ind_catalogue_aftershock)
print(INDEX_DF[ind_catalogue_aftershock_unique])

[ 111  127  128 ... 4855 4856 4859]


In [10]:
print(f"Aftershock removal: {ind_catalogue_aftershock_unique.size} events removed from the catalogue")


Aftershock removal: 1889 events removed from the catalogue


In [11]:
# REMOVAL OF AFTERSHOCK FROM THE CATALOG
df_UPD = deepcopy(df)
df_UPD.drop(index=INDEX_DF[ind_catalogue_aftershock_unique],inplace=True)

In [12]:
print(len(df_UPD))

2708


In [13]:
DISTANCES_UPD = [DISTANCES[k] for k in range(len(df)-1,-1,-1) if k not in ind_catalogue_aftershock_unique]
print(len(DISTANCES_UPD))

2708


In [14]:
#%% Second screening of the catalog for the removal of the FORESHOCK
#% Backward screening

MAGNITUDE_ALL = df_UPD['MwDef'].values
LONG_ALL = df_UPD['LonDef'].values
LAT_ALL = df_UPD['LatDef'].values

INDEX_DF_UPD = df_UPD.index.values

Year_all   = df_UPD['Year'].values
Month_all  = df_UPD['Mo'].values
Day_all    = df_UPD['Da'].values
Hour_all   = df_UPD['Ho'].values
Minute_all = df_UPD['Mi'].values
Second_all = df_UPD['Se'].values

Year_all = Year_all.astype('int')
Month_all = Month_all.astype('int')
Day_all = Day_all.astype('int')
Hour_all = Hour_all.astype('int')
Minute_all = Minute_all.astype('int')
Second_all = Second_all.astype('int')

ind_catalogue_foreshock = []

# CONVERSION OF ALL DATES TO NUMBERS
DateNumber_all = [date_number(Year_all[j],Month_all[j],Day_all[j],Hour_all[j],Minute_all[j],Second_all[j]) for j in range(len(df_UPD))]

for i in range(len(df_UPD)-1,-1,-1):
    # Extraction of the magnitude from the catalog
    Mw = MAGNITUDE_ALL[i]

    # Extraction of epicenter location from the catalog
    LONG = LONG_ALL[i]
    LAT  = LAT_ALL[i]
    
    # DEFINITION OF THE TIME WINDOW ACCORDING TO 3 DIFFERENT FORMULATIONS
    # The final time windows, in days, will be average of the three.
    if Mw >= 6.5:
        # Formulation according to (Gardner and Knopoff 1974)
        DT1 = 10**(0.032*Mw + 2.7389)  # DAYS
        # Formulation according to (Gruenthal)
        DT2=abs(np.exp(-3.95+(0.62+17.32*Mw)**2)) # DAYS
    else:
        # Formulation according to (Gardner and Knopoff 1974)
        DT1 = 10**(0.5409*Mw - 0.547) # DAYS
        # Formulation according to (Gruenthal)
        DT2=10**(2.8+0.024*Mw) # DAYS

    # Formulation according to (Uhrhammer 1986)
    DT3 = np.exp(-2.87+1.235*Mw)
    
    # Average of the three different formluations
    DT = [DT1, DT2, DT3]
    DT = [0 if dt == np.inf else dt for dt in DT]
    DT = max(DT) # DAYS
    
    # DEFINITION OF THE SPATIAL WINDOW ACCORDING TO 3 DIFFERENT FORMULATIONS
    # The final time windows, in days, will be average of the three.
    # Formulation according to (Gardner and Knopoff 1974)
    R1 = 10**(0.1238*Mw+0.983) # km
    # Formulation according to (Gruenthal)
    R2 = np.exp(1.77+(0.037+1.02*Mw)**2) # km
    # Formulation according to (Uhrhammer 1986)
    R3 = np.exp(-1.024+0.804*Mw) # km
    
    # Average of the three different formluations
    R = [R1, R2, R3]
    R = [0 if r > 1000 else r for r in R]
    R = max(R) # km
    
    # CALCULATION OF THE DISTANCE BETWEEN THE CURRENT EPICENTER AND ALL THE
    # OTHER EPICENTERS OF THE OTHER EVENTS
    DISTANCES_i = DISTANCES_UPD[i]

    Year   = Year_all[i]
    Month  = Month_all[i]
    Day    = Day_all[i]
    Hour   = Hour_all[i]
    Minute = Minute_all[i]
    Second = Second_all[i]

    # CALCULATION OF THE CURRENT DAY + TIME WINDWOW
    y=np.floor(DT/365)
    m=np.floor(((DT/365-y)*365)/30)
    d=(((DT/365-y)*365)/30-m)*30

    months_days = [31 if i not in [3, 5, 8, 10] else 30 for i in range(12)]
    months_days[1] = 29 if is_leap_year(int(Year+y)) else 28

    while int(Month+m) > 12 or int(Day+d) > months_days[int(Month+m)-1]:
        
        if int(Month+m) > 12:
            y += 1
            m -= 12
            months_days[1] = 29 if is_leap_year(int(Year+y)) else 28

        if int(Day+d) > months_days[int(Month+m)-1]:
            d -=  months_days[int(Month+m)-1]
            m += 1

    UpperBoundTime = date_number(int(Year+y),int(Month+m),int(Day+d),Hour,Minute,Second)
    
    # IDENTIFICATION OF THE EVENTS FALLING IN THE TIME-SPACE WINDOW HAVING
    # A MAGNITUDE LOWER OR EQUAL THAN THE CURRENT VALUE Mw
  
    index = list()
    [index.append(idx) for idx in range(i-1,-1,-1) if DateNumber_all[idx]>DateNumber_all[i] and DateNumber_all[idx]<=UpperBoundTime and MAGNITUDE_ALL[idx]<Mw and DISTANCES_i[idx]<=R]
    
    # ASSEMBLING THE INDEXES IN A SINGLE MATRIX
    [ind_catalogue_foreshock.append(idx) for idx in index if index != []]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: RuntimeWarning: overflow encountered in exp


In [15]:
# REMOVAL OF REPETING INDEXES
ind_catalogue_foreshock_unique = np.unique(ind_catalogue_foreshock)
print(f"Foreshock removal: {ind_catalogue_foreshock_unique.size} events removed from the catalogue")

Foreshock removal: 0 events removed from the catalogue


In [16]:
# REMOVAL OF FORESHOCK FROM THE CATALOG
df_UPD2 = deepcopy(df_UPD)
if ind_catalogue_foreshock_unique.size > 0:
    df_UPD2.drop(index=INDEX_DF_UPD[ind_catalogue_foreshock_unique],inplace=True)

In [17]:
print(len(df_UPD2))

2708


In [18]:
df_UPD2.index.names = ['Index']
print(df_UPD2.head())
df_UPD2.to_excel(output_catalogue_path, header=True)

       Year   Mo    Da   Ho  ...  DepDef  MwDef             X             Y
Index                        ...                                           
2      1019  4.0   1.0  0.0  ...     NaN   4.63  9.850866e+05  4.569418e+06
3      1044  4.0  19.0  9.0  ...     NaN   4.63  9.850866e+05  4.569418e+06
5      1065  3.0  27.0  0.0  ...     NaN   5.10  5.952487e+05  5.043553e+06
7      1087  9.0  10.0  0.0  ...     NaN   4.86  1.160782e+06  4.583025e+06
8      1091  1.0  27.0  0.0  ...     NaN   5.10  7.884312e+05  4.644411e+06

[5 rows x 12 columns]
